In [4]:


%load_ext autoreload
%autoreload 2

from IPython.display import display
from pprint import pprint

from anthropic import Anthropic

from gdutils.rand import DEFAULT_RANDOM_SEED, set_seeds
from _secrets import CLAUDE_API_KEY

set_seeds(DEFAULT_RANDOM_SEED)

# GREEK_WIKTIONARY_HTML = "/Users/greg/Library/Mobile Documents/3L68KQB4HG~com~readdle~CommonDocuments/Documents/Greek learning/Greek learning - Alexandra/Greek Wiktionary_Frequency lists_Modern Greek_5K Wordlist - Wiktionary, the free dictionary.html"

anthropic_client = Anthropic(api_key=CLAUDE_API_KEY)

ALREADY_WORDS_FILEN = 'already_words.txt'
already_words = open(ALREADY_WORDS_FILEN, 'r').readlines()
already_words = [word.strip() for word in already_words if word.strip()]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import random

from gdutils.llm_utils import generate_gpt_from_template
from prompt_templates import generate_sentence_flashcards

subset_already_words = random.sample(already_words, 20)
print(', '.join(subset_already_words))

context_d = {
    "target_language_name": "Greek",
    "already_words": subset_already_words,
}
out, extra = generate_gpt_from_template(
    client=anthropic_client,
    prompt_template_var="generate_sentence_flashcards",
    context_d=context_d,
    response_json=True,
    verbose=2,
)
print()
print('----')

for x in out["sentences"]:
    print(x["sentence"])
    print(x["translation"])
    print(x["lemma_words"])
    print()


αρδευτικός, κατανόηση, τεράστιος, πειρατής, σκλάβος, θέληση, ωκεανός, φυλακίζω, σηκώνω, συγκρούομαι, μεταμορφώνομαι, μπολ, σκαρφαλώνω, προειδοποιώ, τρέχω, συντήρηση, σμιλεύω, διαλέγω, τέρας, ζήλευε
Called GPT on 'generate_sentence_flashcards', context keys ['target_language_name', 'already_words']

----
Οι πειρατές του ωκεανού συγκρούστηκαν με το τεράστιο πλοίο.
The pirates of the ocean clashed with the huge ship.
['πειρατής', 'ωκεανός', 'συγκρούομαι', 'τεράστιος']

Ο σκλάβος σκαρφάλωσε στον τοίχο για να δραπετεύσει από τη φυλακή.
The slave climbed the wall to escape from prison.
['σκλάβος', 'σκαρφαλώνω', 'φυλακίζω']

Έτρεξα να προειδοποιήσω τους φίλους μου για το τέρας.
I ran to warn my friends about the monster.
['τρέχω', 'προειδοποιώ', 'τέρας']

Το αρδευτικό σύστημα χρειάζεται συντήρηση.
The irrigation system needs maintenance.
['αρδευτικός', 'συντήρηση']



In [49]:
import json
import time
from typing import Optional

from audio_utils import generate_txt_audio, play_txt_audio
from config import OUT_DIRN
from lang_utils import get_language_name

target_language_code = "el"
target_language_name = get_language_name(target_language_code)
SENTENCES_DIRN = OUT_DIRN / target_language_name / "sentences"
SENTENCES_DIRN.mkdir(parents=True, exist_ok=True)


def generate_sentence(
    target_language_code: str,
    sentence: str,
    translation: str,
    lemma_words: Optional[list]=None,
    should_play: bool = False
):
    extra = locals()
    if sentence.endswith("."):
        sentence = sentence[:-1]
    mp3_filen = SENTENCES_DIRN / (sentence + ".mp3")

    txt_tgt_mp3_filen = generate_txt_audio(
        txt=sentence,
        should_add_delays=True,
        should_play=should_play,
        mp3_dirn=SENTENCES_DIRN,
        mp3_filen=mp3_filen,
        # overwrite_already_exists=True,
        verbose=1,
    )
    json_filen = SENTENCES_DIRN / (sentence + ".json")
    json_filen.write_text(json.dumps(extra, indent=2))

    extra.update(
        {
            "mp3_filen": str(mp3_filen),
            "json_filen": str(json_filen),
            "sentence": sentence,
            "translation": translation,
            "lemma_words": lemma_words,
        }
    )
    return txt_tgt_mp3_filen, extra


def test_random(extras: list[dict], wait_secs: float = 3):
    cur = random.choice(extras)
    txt = cur["sentence"]
    translation = cur["translation"]
    mp3_filen = cur["mp3_filen"]
    print("Playing...",)
    play_txt_audio(txt, mp3_filen=mp3_filen)
    time.s leep(wait_secs)
    print(txt)
    print(cur["lemma_words"])
    time.sleep(wait_secs)
    print(translation)

In [51]:
# get a list of .json files in SENTENCES_DIRN
json_files = list(SENTENCES_DIRN.glob("*.json"))

for json_filen in json_files:
    extra = json.loads(json_filen.read_text())

for sentence in out["sentences"]:
    mp3_filen, extra = generate_sentence(
        target_language_code=target_language_code,
        sentence=sentence["sentence"],
        translation=sentence["translation"],
        lemma_words=sentence["lemma_words"],
    )
    extras.append(extra)

Generating MP3 file for Οι πειρατές του ωκεανού συγκρούστηκαν με το τεράστιο πλοίο -> /Users/greg/Dropbox/dev/spideryarn/241103_vocab/out/Greek/sentences/Οι πειρατές του ωκεανού συγκρούστηκαν με το τεράστιο πλοίο.mp3
Generating MP3 file for Ο σκλάβος σκαρφάλωσε στον τοίχο για να δραπετεύσει από τη φυλακή -> /Users/greg/Dropbox/dev/spideryarn/241103_vocab/out/Greek/sentences/Ο σκλάβος σκαρφάλωσε στον τοίχο για να δραπετεύσει από τη φυλακή.mp3
Generating MP3 file for Έτρεξα να προειδοποιήσω τους φίλους μου για το τέρας -> /Users/greg/Dropbox/dev/spideryarn/241103_vocab/out/Greek/sentences/Έτρεξα να προειδοποιήσω τους φίλους μου για το τέρας.mp3
Generating MP3 file for Το αρδευτικό σύστημα χρειάζεται συντήρηση -> /Users/greg/Dropbox/dev/spideryarn/241103_vocab/out/Greek/sentences/Το αρδευτικό σύστημα χρειάζεται συντήρηση.mp3


In [55]:
test_random(extras)

Playing...
Έτρεξα να προειδοποιήσω τους φίλους μου για το τέρας
['τρέχω', 'προειδοποιώ', 'τέρας']
I ran to warn my friends about the monster.
